In [1]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import certifi
import urllib3
import warnings
warnings.filterwarnings("ignore")


In [2]:
import urllib3
from bs4 import BeautifulSoup
from requests.packages.urllib3.exceptions import InsecureRequestWarning


def get_all_links(url):

# Suppress only the InsecureRequestWarning
    requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

    http = urllib3.PoolManager(cert_reqs='CERT_NONE')
    response = http.request('GET', url)

    # Check if the request was successful
    if response.status != 200:  # تأكد من حالة الاستجابة
        print(f"Failed to retrieve the page. Status code: {response.status}")
        return []

    # Parse the HTML content of the page
    soup = BeautifulSoup(response.data.decode('utf-8'), 'html.parser')  

    # Find all links
    links = []
    for link in soup.find_all('a'):
        href = link.get('href')
        if href:
            links.append(href)

    return links

# Example usage
url = 'https://neoma-bs.com/'  
all_links = get_all_links(url)
print("All links found:")
for link in all_links:
    print(link)


All links found:
https://www.efmdglobal.org/accreditations/business-schools/equis/
https://www.aacsb.edu/
https://www.associationofmbas.com/
https://neoma-bs.com/contact
#
https://welcome.neoma-bs.com
https://neoma-bs.com/neoma-s-world/
https://startuplab.neoma-bs.fr/en/
https://www.fondation-neoma.fr/en/
http://myneomagora.neoma-bs.fr
http://library.neoma-bs.fr/en/?_=1062393260
https://www.facebook.com/NEOMAbs/
https://twitter.com/neomabs
https://www.instagram.com/neomabs/
https://www.tiktok.com/@neomabs?lang=fr
https://www.youtube.com/user/NEOMAbsWebTV
https://www.linkedin.com/edu/school?id=12416
https://neoma-bs.fr/
https://neoma-bs.com/
#
https://neoma-bs.com/about-neoma/about-neoma-business-school/
https://neoma-bs.com/about-neoma/about-neoma-business-school/ambition/
https://neoma-bs.com/about-neoma/about-neoma-business-school/values/
https://neoma-bs.com/about-neoma/about-neoma-business-school/be-passionate-shape-the-future/
https://neoma-bs.com/about-neoma/about-neoma-business-

In [3]:
# with open("URL.txt",'w')as f:
#     for link in all_links:
#         f.write(link + "\n")
        


In [4]:
import os

In [5]:
link_without_exe = []
def link_exe(linkes):
    for link in linkes:
        exe,_ =  os.path.splitext(link)
        link_without_exe.append(exe)
    return link_without_exe


In [6]:
linkes = link_exe(all_links)
linkes

['https://www.efmdglobal.org/accreditations/business-schools/equis/',
 'https://www.aacsb.edu/',
 'https://www.associationofmbas.com/',
 'https://neoma-bs.com/contact',
 '#',
 'https://welcome.neoma-bs',
 'https://neoma-bs.com/neoma-s-world/',
 'https://startuplab.neoma-bs.fr/en/',
 'https://www.fondation-neoma.fr/en/',
 'http://myneomagora.neoma-bs',
 'http://library.neoma-bs.fr/en/?_=1062393260',
 'https://www.facebook.com/NEOMAbs/',
 'https://twitter.com/neomabs',
 'https://www.instagram.com/neomabs/',
 'https://www.tiktok.com/@neomabs?lang=fr',
 'https://www.youtube.com/user/NEOMAbsWebTV',
 'https://www.linkedin.com/edu/school?id=12416',
 'https://neoma-bs.fr/',
 'https://neoma-bs.com/',
 '#',
 'https://neoma-bs.com/about-neoma/about-neoma-business-school/',
 'https://neoma-bs.com/about-neoma/about-neoma-business-school/ambition/',
 'https://neoma-bs.com/about-neoma/about-neoma-business-school/values/',
 'https://neoma-bs.com/about-neoma/about-neoma-business-school/be-passionate-sh

In [7]:
final_link = []
pattern = "https://neoma"

for link in linkes:
    if pattern in link:
        final_link.append(link)

In [8]:
final_link

['https://neoma-bs.com/contact',
 'https://neoma-bs.com/neoma-s-world/',
 'https://neoma-bs.fr/',
 'https://neoma-bs.com/',
 'https://neoma-bs.com/about-neoma/about-neoma-business-school/',
 'https://neoma-bs.com/about-neoma/about-neoma-business-school/ambition/',
 'https://neoma-bs.com/about-neoma/about-neoma-business-school/values/',
 'https://neoma-bs.com/about-neoma/about-neoma-business-school/be-passionate-shape-the-future/',
 'https://neoma-bs.com/about-neoma/about-neoma-business-school/key-figures/',
 'https://neoma-bs.com/about-neoma/about-neoma-business-school/rankings/',
 'https://neoma-bs.com/about-neoma/corporate-social-responsibility-csr/',
 'https://neoma-bs.com/about-neoma/corporate-social-responsibility-csr/our-csr-policy/',
 'https://neoma-bs.com/about-neoma/corporate-social-responsibility-csr/our-csr-actions/',
 'https://neoma-bs.com/about-neoma/innovation/',
 'https://neoma-bs.com/about-neoma/innovation/digital-transformation/',
 'https://neoma-bs.com/faculty-and-res

In [9]:
final_link = set(final_link)

In [10]:
# with open("FinalURL.txt",'w')as f:
#     for link in final_link:
#         f.write(link + "\n")
        

In [10]:
import os
import certifi
import time
os.environ['REQUESTS_CA_BUNDLE'] = certifi.where()

from langchain.document_loaders import UnstructuredURLLoader
rs= []
for link in final_link:
    r=requests.get(link, verify=False)
    rs.append(r.content)

In [44]:
rs[0]

b'</li></li></li><!DOCTYPE html>\n<!--[if lt IE 9]><html class="no-js no-svg ie lt-ie9 lt-ie8 lt-ie7" lang="en-GB"> <![endif]-->\n<!--[if IE 9]><html class="no-js no-svg ie ie9 lt-ie9 lt-ie8" lang="en-GB"> <![endif]-->\n<!--[if gt IE 9]><!--><html class="no-js no-svg" lang="en-GB"> <!--<![endif]-->\n    <head>\n        <meta http-equiv="content-type" content="text/html; charset=UTF-8">\n    <meta http-equiv="X-UA-Compatible" content="IE=edge">\n    <meta name="viewport" content="width=device-width, initial-scale=1">\n    <link rel="stylesheet" href="https://neoma-bs.com/wp-content/themes/theme_ci/style.css" type="text/css" media="screen" />\n    <link rel="author" href="https://neoma-bs.com/wp-content/themes/theme_ci/humans.txt" />\n    <link rel="pingback" href="https://neoma-bs.com/xmlrpc.php" />\n    <link rel="profile" href="http://gmpg.org/xfn/11">\n\n    <link rel="apple-touch-icon" sizes="180x180" href="https://neoma-bs.com/wp-content/themes/theme_ci/favicons/apple-touch-icon.pn

In [22]:
import certifi
print(certifi.where())  # Use this path for REQUESTS_CA_BUNDLE



/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/certifi/cacert.pem


In [11]:
rs[0]

b'</li></li></li><!DOCTYPE html>\n<!--[if lt IE 9]><html class="no-js no-svg ie lt-ie9 lt-ie8 lt-ie7" lang="en-GB"> <![endif]-->\n<!--[if IE 9]><html class="no-js no-svg ie ie9 lt-ie9 lt-ie8" lang="en-GB"> <![endif]-->\n<!--[if gt IE 9]><!--><html class="no-js no-svg" lang="en-GB"> <!--<![endif]-->\n    <head>\n        <meta http-equiv="content-type" content="text/html; charset=UTF-8">\n    <meta http-equiv="X-UA-Compatible" content="IE=edge">\n    <meta name="viewport" content="width=device-width, initial-scale=1">\n    <link rel="stylesheet" href="https://neoma-bs.com/wp-content/themes/theme_ci/style.css" type="text/css" media="screen" />\n    <link rel="author" href="https://neoma-bs.com/wp-content/themes/theme_ci/humans.txt" />\n    <link rel="pingback" href="https://neoma-bs.com/xmlrpc.php" />\n    <link rel="profile" href="http://gmpg.org/xfn/11">\n\n    <link rel="apple-touch-icon" sizes="180x180" href="https://neoma-bs.com/wp-content/themes/theme_ci/favicons/apple-touch-icon.pn

In [39]:
from langchain_community.document_loaders import UnstructuredHTMLLoader

loadder = UnstructuredHTMLLoader("./data_html.html")

data = loadder.load()

In [40]:
data

[Document(metadata={'source': './data_html.html'}, page_content='Home\n\nApprenticeship Tax\n\nApprenticeship Tax\n\nCompanies subject to French apprenticeship tax have the possibility to choose NEOMA as recipient.\n\nThis choice means working with us towards achieving our main goal: training your future employees through pedagogic innovation and academic excellence.\n\nLearn more (for companies subject to French apprenticeship tax)\n\nContact us\n\nIsabelle BALDARI-PLAISANT\n\nisabelle.baldari@neoma-bs.fr\n\n02 32 82 58 46')]

In [41]:
from langchain_core.documents import Document
from langchain_community.vectorstores import Chroma


In [42]:
documents = []

for line in data:
    documents.append(Document(line.page_content))

In [43]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

embedding_function = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

vectorstore = Chroma.from_documents(documents, embedding_function)

retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 6}
)


C:\Users\StarNet\AppData\Local\Temp\ipykernel_6080\1781988043.py:5: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  embedding_function = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
